In [1]:
from ema_workbench.em_framework.optimization import (
    ArchiveLogger, 
    EpsilonProgress, 
    to_problem,
    epsilon_nondominated
)

from ema_workbench import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
    Model,
    Scenario,
    CategoricalParameter,
    ArrayOutcome,
    ScalarOutcome,
    IntegerParameter,
    RealParameter,
    Policy,
    MultiprocessingEvaluator,
)

from dike_model_function import DikeNetwork  # @UnresolvedImport
from ema_workbench.util import ema_logging

import pandas as pd
from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

## Hard Coded Model -- To be replaced by problem_formulation

In [2]:
def sum_over(*args):
    numbers = []
    for entry in args:
        try:
            value = sum(entry)
        except TypeError:
            value = entry
        numbers.append(value)

    return sum(numbers)

In [16]:
model_a4, planning_steps = get_model_for_problem_formulation('A4 Only')
problem_a4 = to_problem(model_a4, searchover="levers")

# Model with 13 outcomes, neccesary to apply the constraits
model_all, planning_steps = get_model_for_problem_formulation('All Dikes')
problem_all = to_problem(model_all, searchover="levers")


## Read Results

In [5]:
scenarios_df = pd.read_csv('./output/selected_scenarios.csv')
scenarios_df.head()

,Run ID,A0_ID_flood_wave_shape,A1_Bmax,A1_Brate,A1_pfail,A2_Bmax,A2_Brate,A2_pfail,A3_Bmax,A3_Brate,A3_pfail,A4_Bmax,A4_Brate,A4_pfail,A5_Bmax,A5_Brate,A5_pfail,discount_rate_0,discount_rate_1,discount_rate_2
0,97010,123,67.745865,10.0,0.182689,279.534112,10.0,0.689970,47.745028,1.5,0.076163,56.574990,1.5,0.322450,116.530208,1.5,0.876601,4.5,1.5,1.5
1,60187,128,278.641940,1.0,0.893895,243.623822,10.0,0.747435,287.003137,1.5,0.594750,237.609389,1.0,0.860543,97.712446,1.0,0.536116,1.5,1.5,3.5
2,67375,60,48.962139,1.5,0.066552,299.157411,10.0,0.873473,37.724188,10.0,0.413907,177.710030,1.5,0.113153,299.944753,1.0,0.775375,2.5,2.5,1.5
3,18777,68,128.130163,10.0,0.791207,339.425606,10.0,0.849394,31.308689,10.0,0.968231,164.774507,10.0,0.109501,307.969700,10.0,0.583670,2.5,1.5,1.5


In [6]:
scenarios = [str(int(row['Run ID'])) for _, row in scenarios_df.iterrows()]
scenarios.append('Reference')
scenarios

['97010', '60187', '67375', '18777', 'Reference']

In [19]:
# Assumes seeds are sequential starting at 0 -- could be adapted to arbitrary
results = {}
convergences = {}
archives = {}

for scenario in scenarios:
    results[scenario] = []
    convergences[scenario] = []
    archives[scenario] = []
    for seed in range(5):
        # Results and Convergences
        fn_head = './output/DIRECTED_SEARCH__'
        fn_tail = f'__scen{scenario}__seed{seed}.csv'

        res = pd.read_csv(fn_head + 'results' + fn_tail, index_col=0)
        results[scenario].append(res)

        conv = pd.read_csv(fn_head + 'convergence' + fn_tail, index_col=0)
        convergences[scenario].append(conv)

        # Archives
        fn_head = './archives/DIRECTED_SEARCH__'
        fn_tail = f'__scen{scenario}__seed{seed}.tar.gz'
        arch = ArchiveLogger.load_archives(fn_head + 'archive' + fn_tail)
        archives[scenario].append(arch)


## Filtering Out Pareto-Dominated Policies
(within each scenario)

In [29]:
policy_sets = {}
epsilon = [100, 0.01, 100, 100, 0.01]
for scenario in scenarios:
    df = epsilon_nondominated(results[scenario], epsilon, problem_a4)
    # df = df.drop([o.name for o in model_a4.outcomes], axis=1)
    policy_sets[scenario] = df
    n_policies = df.shape[0]
    print(f"Scenario {scenario} has {n_policies} non-dominated policies")
    # TODO: Write pareto-nondominated policies to a file

Scenario 97010 has 55 non-dominated policies
Scenario 60187 has 9 non-dominated policies
Scenario 67375 has 65 non-dominated policies
Scenario 18777 has 24 non-dominated policies
Scenario Reference has 33 non-dominated policies


## Convergence Plots

In [33]:
policy_sets['18777'].head()

,EWS_DaysToThreat,rfr_0_t0,rfr_0_t1,rfr_0_t2,rfr_1_t0,rfr_1_t1,rfr_1_t2,rfr_2_t0,rfr_2_t1,rfr_2_t2,...,A4_DikeIncrease_t1,A4_DikeIncrease_t2,A5_DikeIncrease_t0,A5_DikeIncrease_t1,A5_DikeIncrease_t2,A4_Expected_Annual_Damage,A4_Expected_Number_of_Deaths,Total_Infrastructure_Costs,Total_Expected_Annual_Damage,Total_Expected_Number_of_Deaths
0,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,1.177718e+08,1.804251e+07,0.001280
1,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,1.554910e+08,0.000000e+00,0.000000
2,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,1.486195e+08,2.553720e+06,0.000188
3,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,1.159246e+08,1.935598e+07,0.001357
4,3,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,1.504667e+08,9.257982e+05,0.000091


In [31]:
for scenario in scenarios:
    pols = policy_sets[scenario]
    print(pols.columns)
    hv = HypervolumeMetric(pols, problem_a4)
    gd = GenerationalDistanceMetric(pols, problem_a4, d=1)
    ei = EpsilonIndicatorMetric(pols, problem_a4)
    ig = InvertedGenerationalDistanceMetric(pols, problem_a4, d=1)
    sm = SpacingMetric(problem_a4)

    metrics_by_seed = []

    for archive in archives:
        metrics = []
        for nfe, a in archive.items():
            scores = {
                "generational_distance": gd.calculate(a),
                "hypervolume": hv.calculate(a),
                "epsilon_indicator": ei.calculate(a),
                "inverted_gd": ig.calculate(a),
                "spacing": sm.calculate(a),
                "nfe": int(nfe),
            }
            metrics.append(scores)
        metrics = pd.DataFrame.from_dict(metrics)

        # sort metrics by number of function evaluations
        metrics.sort_values(by="nfe", inplace=True)
        metrics_by_seed.append(metrics)
    
    fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

    ax1, ax2, ax3, ax4, ax5, ax6 = axes

    for metrics, convergence in zip(metrics_by_seed, convergences):
        ax1.plot(metrics.nfe, metrics.hypervolume)
        ax1.set_ylabel("hypervolume")

        ax2.plot(convergence.nfe, convergence.epsilon_progress)
        ax2.set_ylabel("$\epsilon$ progress")

        ax3.plot(metrics.nfe, metrics.generational_distance)
        ax3.set_ylabel("generational distance")

        ax4.plot(metrics.nfe, metrics.epsilon_indicator)
        ax4.set_ylabel("epsilon indicator")

        ax5.plot(metrics.nfe, metrics.inverted_gd)
        ax5.set_ylabel("inverted generational\ndistance")

        ax6.plot(metrics.nfe, metrics.spacing)
        ax6.set_ylabel("spacing")

    ax6.set_xlabel("nfe")
    ax1.title(f'Scenario {scenario} Convergence Graphs')

    sns.despine(fig)

    plt.show()

Index(['EWS_DaysToThreat', 'rfr_0_t0', 'rfr_0_t1', 'rfr_0_t2', 'rfr_1_t0',
       'rfr_1_t1', 'rfr_1_t2', 'rfr_2_t0', 'rfr_2_t1', 'rfr_2_t2', 'rfr_3_t0',
       'rfr_3_t1', 'rfr_3_t2', 'rfr_4_t0', 'rfr_4_t1', 'rfr_4_t2',
       'A1_DikeIncrease_t0', 'A1_DikeIncrease_t1', 'A1_DikeIncrease_t2',
       'A2_DikeIncrease_t0', 'A2_DikeIncrease_t1', 'A2_DikeIncrease_t2',
       'A3_DikeIncrease_t0', 'A3_DikeIncrease_t1', 'A3_DikeIncrease_t2',
       'A4_DikeIncrease_t0', 'A4_DikeIncrease_t1', 'A4_DikeIncrease_t2',
       'A5_DikeIncrease_t0', 'A5_DikeIncrease_t1', 'A5_DikeIncrease_t2',
       'A4_Expected_Annual_Damage', 'A4_Expected_Number_of_Deaths',
       'Total_Infrastructure_Costs', 'Total_Expected_Annual_Damage',
       'Total_Expected_Number_of_Deaths'],
      dtype='object')


PlatypusError: objective with empty range

In [60]:
pol_set = policy_sets['18777']
levers_quick = [l.name for l in model_a4.levers if l.name.split('_')[-1] == 't0'] + ['EWS_DaysToThreat']
pol_set[levers_quick].head()

,rfr_0_t0,rfr_1_t0,rfr_2_t0,rfr_3_t0,rfr_4_t0,A1_DikeIncrease_t0,A2_DikeIncrease_t0,A3_DikeIncrease_t0,A4_DikeIncrease_t0,A5_DikeIncrease_t0,EWS_DaysToThreat
0,0,0,0,0,0,0,2,2,8,5,3
1,0,0,0,0,0,1,3,2,8,5,3
2,0,0,0,0,0,1,2,2,7,5,3
3,0,0,0,0,0,0,2,2,7,5,3
4,0,0,0,0,0,1,2,2,8,5,3


In [44]:
outcomes_of_interest = ['A4_Expected_Annual_Damage', 'A4_Expected_Number_of_Deaths',
                        'Total_Expected_Annual_Damage', 'Total_Expected_Number_of_Deaths',
                        'Total_Infrastructure_Costs']

In [47]:
pol_set_outcomes = pol_set[outcomes_of_interest]
pol_set_outcomes.head()

,A4_Expected_Annual_Damage,A4_Expected_Number_of_Deaths,Total_Expected_Annual_Damage,Total_Expected_Number_of_Deaths,Total_Infrastructure_Costs
0,0,0,1.804251e+07,0.001280,1.177718e+08
1,0,0,0.000000e+00,0.000000,1.554910e+08
2,0,0,2.553720e+06,0.000188,1.486195e+08
3,0,0,1.935598e+07,0.001357,1.159246e+08
4,0,0,9.257982e+05,0.000091,1.504667e+08


In [51]:
for scenario in scenarios:
    pol_set = policy_sets[scenario]
    pol_set_outcomes = pol_set[outcomes_of_interest]
    print('Scenario:', scenario)
    for outcome in outcomes_of_interest:
        print(' :: Outcome:', outcome)
        print(' :::: Max:', pol_set_outcomes[outcome].max())
        print(' :::: Min:', pol_set_outcomes[outcome].min())

Scenario: 97010
 :: Outcome: A4_Expected_Annual_Damage
 :::: Max: 0
 :::: Min: 0
 :: Outcome: A4_Expected_Number_of_Deaths
 :::: Max: 0
 :::: Min: 0
 :: Outcome: Total_Expected_Annual_Damage
 :::: Max: 4929408955.804385
 :::: Min: 0.0
 :: Outcome: Total_Expected_Number_of_Deaths
 :::: Max: 0.5617244316908736
 :::: Min: 0.0
 :: Outcome: Total_Infrastructure_Costs
 :::: Max: 193903705.7137209
 :::: Min: 0.0
Scenario: 60187
 :: Outcome: A4_Expected_Annual_Damage
 :::: Max: 0
 :::: Min: 0
 :: Outcome: A4_Expected_Number_of_Deaths
 :::: Max: 0
 :::: Min: 0
 :: Outcome: Total_Expected_Annual_Damage
 :::: Max: 223725043.06637296
 :::: Min: 0.0
 :: Outcome: Total_Expected_Number_of_Deaths
 :::: Max: 0.0295595554778626
 :::: Min: 0.0
 :: Outcome: Total_Infrastructure_Costs
 :::: Max: 103688774.4749054
 :::: Min: 0.0
Scenario: 67375
 :: Outcome: A4_Expected_Annual_Damage
 :::: Max: 0
 :::: Min: 0
 :: Outcome: A4_Expected_Number_of_Deaths
 :::: Max: 0
 :::: Min: 0
 :: Outcome: Total_Expected_Annu

In [38]:
from ema_workbench.em_framework.optimization import rebuild_platypus_population

In [43]:
hv = HypervolumeMetric(pol_set, problem_a4)

PlatypusError: objective with empty range

In [ ]:
min, max = normalize(pol_set)
::


## Filtering Policy Sets According to Constraints

In [ ]:
policies_to_evaluate = []

for scenario in scenarios:
    policies = policy_sets[scenario]
    for _, policy in policies.iterrows():
        policies_to_evaluate.append(Policy(str(_), **policy.to_dict()))
    # baseline policy, needed for for relative constraits
    zero_policy = {"EWS_DaysToThreat": 0}
    for i in range(5):
        dike = "A" + str(i+1)
        zero_policy.update({f"{dike}_DikeIncrease_t{n}": 0 for n in planning_steps})
        zero_policy.update({f"rfr_{i}_t{n}": 0 for n in planning_steps})

    p_ref = Policy("Reference Policy", **zero_policy)
    policies_to_evaluate.append(p_ref)

    # pass the policies list to EMA workbench experiment runs
    with MultiprocessingEvaluator(model_all) as evaluator:
        results = evaluator.perform_experiments(ref_scenario, policies_to_evaluate)
    